In [10]:
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np

In [11]:
import pandas as pd 
traindata = pd.read_csv("train.csv",  encoding='utf8' )
del traindata['Unnamed: 0']
traindata.head()

,id,text,label
0,547530,Beşiktaş'ın eski teknik direktörü Slaven Bilic...,spor
1,42683,"14 Şubat, Katolik Kilisesi’nin azizlerinden St...",yazarlar
2,770925,İstanbul Kağıthane’de gece yarısı sokakta oyun...,video
3,473224,Ziraat Türkiye Kupası maçında deplasmanda Amed...,video
4,44833,"Eski Brezilyalı yıldız oyuncu Ronaldo, Brezily...",spor


Getting train and test data

In [12]:
testdata = pd.read_csv("test.csv",  encoding='utf8' )
del testdata['Unnamed: 0']

#Consider the input values for X and y
X_train = traindata['text']
y_train = traindata['label']

X_test = testdata['text']
y_test = testdata['label']


Importing Neccessary Libraries 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from time import time
from pprint import pprint

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words("turkish"))

Grid Search Naive Bayes

In [ ]:
pipeline = Pipeline(steps=[
    ('tfidf', TfidfVectorizer()),
    ('multinomialnb', MultinomialNB()),
])

parameters = {
    'tfidf__ngram_range': ((1,1), (1,2)), 
    'tfidf__stop_words': (None, stop_words),
    'tfidf__use_idf': (True, False),
    'multinomialnb__alpha': (0.5, 1.0),
    'multinomialnb__fit_prior': (True, False),
}

grid_search = GridSearchCV(pipeline, parameters, scoring='accuracy', verbose=1)

grid_search.fit(X_train, y_train)


print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()

for param_name in sorted(parameters.keys()):
  print("\t%s: %r" % (param_name, best_parameters[param_name]))

print(classification_report(grid_search.predict(X_test), y_test))
print ('\n')
print(confusion_matrix(y_test, grid_search.predict(X_test)))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 160 out of 160 | elapsed: 14.1min finished


Best score: 0.746
Best parameters set:
	multinomialnb__alpha: 0.5
	multinomialnb__fit_prior: False
	tfidf__ngram_range: (1, 1)
	tfidf__stop_words: {'defa', 'gibi', 'nasıl', 'niçin', 'tüm', 'mu', 'sanki', 'siz', 'ama', 'her', 'hiç', 'ne', 'az', 'şey', 'hep', 'ya', 'ile', 'birkaç', 'kez', 'eğer', 'kim', 'mü', 'biri', 'aslında', 'belki', 'diye', 'hepsi', 'nereye', 'en', 'bu', 'da', 'hem', 'acaba', 'çünkü', 'yani', 'veya', 'nerde', 've', 'çok', 'mı', 'için', 'nerede', 'niye', 'ise', 'ki', 'de', 'neden', 'birşey', 'bazı', 'biz', 'daha', 'şu', 'o'}
	tfidf__use_idf: True
              precision    recall  f1-score   support

       dunya       0.81      0.81      0.81       394
        spor       0.95      0.90      0.92       405
     turkiye       0.70      0.60      0.65       490
       video       0.26      0.92      0.41       117
    yazarlar       0.97      0.64      0.77       594

    accuracy                           0.73      2000
   macro avg       0.74      0.78      0.71      

Grid Search Logistic Regression Comparing Solvers

In [17]:
pipeline = Pipeline(steps=[
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression(max_iter= 500)),
])

parameters = {
    'tfidf__stop_words': (None, stop_words),
    'clf__penalty' : ['l2'],
    'clf__C' : (1.0, 10),
    'clf__solver' : ['lbfgs', 'liblinear']
}

grid_search2 = GridSearchCV(pipeline, parameters, scoring='accuracy', verbose=1)

grid_search2.fit(X_train, y_train)


print("Best score: %0.3f" % grid_search2.best_score_)
print("Best parameters set:")
best_parameters = grid_search2.best_estimator_.get_params()

for param_name in sorted(parameters.keys()):
  print("\t%s: %r" % (param_name, best_parameters[param_name]))

print(classification_report(grid_search2.predict(X_test), y_test))
print ('\n')
print(confusion_matrix(y_test, grid_search2.predict(X_test)))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed: 11.0min finished


Best score: 0.843
Best parameters set:
	clf__C: 10
	clf__penalty: 'l2'
	clf__solver: 'liblinear'
	tfidf__stop_words: None
              precision    recall  f1-score   support

       dunya       0.87      0.82      0.84       422
        spor       0.95      0.94      0.95       388
     turkiye       0.71      0.79      0.75       382
       video       0.76      0.79      0.78       394
    yazarlar       0.95      0.90      0.93       414

    accuracy                           0.85      2000
   macro avg       0.85      0.85      0.85      2000
weighted avg       0.85      0.85      0.85      2000



[[344   3  17  21  10]
 [  6 366   4   3   5]
 [ 37   2 300  57  25]
 [ 31  11  54 311   1]
 [  4   6   7   2 373]]


Grid Search Logistic Regression Comparing Penalties

In [19]:
pipeline = Pipeline(steps=[
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression(max_iter= 500)),
])

parameters = {
    'tfidf__stop_words': (None, stop_words),
    'clf__penalty' : ['l1','l2'],
    'clf__C' : (0.1, 1),
    'clf__solver' : ['liblinear']
}

grid_search2 = GridSearchCV(pipeline, parameters, scoring='accuracy', verbose=1)

grid_search2.fit(X_train, y_train)


print("Best score: %0.3f" % grid_search2.best_score_)
print("Best parameters set:")
best_parameters = grid_search2.best_estimator_.get_params()

for param_name in sorted(parameters.keys()):
  print("\t%s: %r" % (param_name, best_parameters[param_name]))

print(classification_report(grid_search2.predict(X_test), y_test))
print ('\n')
print(confusion_matrix(y_test, grid_search2.predict(X_test)))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  3.3min finished


Best score: 0.843
Best parameters set:
	clf__C: 10
	clf__penalty: 'l2'
	clf__solver: 'liblinear'
	tfidf__stop_words: None
              precision    recall  f1-score   support

       dunya       0.87      0.82      0.84       422
        spor       0.95      0.94      0.95       388
     turkiye       0.71      0.79      0.75       382
       video       0.76      0.79      0.78       394
    yazarlar       0.95      0.90      0.93       414

    accuracy                           0.85      2000
   macro avg       0.85      0.85      0.85      2000
weighted avg       0.85      0.85      0.85      2000



[[344   3  17  21  10]
 [  6 366   4   3   5]
 [ 37   2 300  57  25]
 [ 31  11  54 311   1]
 [  4   6   7   2 373]]
